In [74]:
import numpy as np
from scipy.optimize import fsolve

In [179]:
def bond_PV(rates):
    PV = 0
    tree_prices = list(reversed([[0]*(i+1) for i in range(len(rates))])) ## back to front
    rates = list(reversed(rates))
    for i in range(len(tree_prices[0])):
        tree_prices[0][i] = 1/(1+rates[0][i])
        
    for i in range(1,len(tree_prices)):
        for j in range(len(tree_prices[i])):
            tree_prices[i][j] = 0.5*(tree_prices[i-1][j] + tree_prices[i-1][j+1])/(1+rates[i][j])
    return tree_prices[-1][-1],tree_prices

def append_new_rates(ts_curr,m,vol):
    ts_new = [0]*(len(ts_curr[-1])+1) ### create next layer in the tree
    for i in range(len(ts_new)):
        ts_new[i] = ts_curr[-1][0] + m - 2*i*vol
    return ts_curr + [ts_new]

def append_BDT_rates(ts_curr,m,vol):
    ts_new = [0]*(len(ts_curr[-1])+1) ### create next layer in the tree
    for i in range(len(ts_new)):
        ts_new[i] = ts_curr[-1][0]*np.exp(m - 2*i*vol)
    return ts_curr + [ts_new]

class Trees:
    def __init__(self,bond_prices):
        self.bond_prices = bond_prices
    
    def Ho_Lee(self,vol):
        bond_prices = self.bond_prices
        
        def func(m,ts_curr,targ_price):
            ts_optim = append_new_rates(ts_curr,m,vol)
            expec_q,_ = bond_PV(ts_optim)
            return targ_price - expec_q
            
        calib_rates = [[1/bond_prices[0] - 1]]
        guess = 0.05
        
        for i in range(1,len(bond_prices)):
            calib_m = fsolve(func,guess,args=(calib_rates,bond_prices[i]))
            calib_rates = append_new_rates(calib_rates,calib_m,vol)
        return calib_rates

    def BDT(self,vols):
        bond_prices = self.bond_prices
        
        def func(inputs,ts_curr,targ_price,targ_vol):
            m,vol = inputs
            ts_optim = append_BDT_rates(ts_curr,m,vol)
            expec_q,prices = bond_PV(ts_optim)
            yu = (1/prices[-2][0])**(1/(len(ts_optim)-1)) - 1
            yd = (1/prices[-2][1])**(1/(len(ts_optim)-1)) - 1
            expec_vol = 0.5*np.log(yu/yd)
            return [targ_price - expec_q, targ_vol - expec_vol]
            
        calib_rates = [[1/bond_prices[0] - 1]]
        price_guess = 0.05
        vol_guess = 0.04
        
        for i in range(1,len(bond_prices)):
            calib_m,calib_vol = fsolve(func,[price_guess, vol_guess],args=(calib_rates,bond_prices[i],vols[i-1]))
            calib_rates = append_BDT_rates(calib_rates,calib_m,calib_vol)
        return calib_rates

In [180]:
bond_yields = [0.05,0.055,0.057,0.059,0.06,0.061]
bond_prices = [1/(1+x)**(i+1) for (i,x) in enumerate(bond_yields)]
#bond_prices = [95,90,85]
Trees_obj = Trees(bond_prices)
print(Trees_obj.Ho_Lee(0.015))

[[0.050000000000000044], [array([0.07523603]), array([0.04523603])], [array([0.09164759]), array([0.06164759]), array([0.03164759])], [array([0.11129248]), array([0.08129248]), array([0.05129248]), array([0.02129248])], [array([0.12612463]), array([0.09612463]), array([0.06612463]), array([0.03612463]), array([0.00612463])], [array([0.14418405]), array([0.11418405]), array([0.08418405]), array([0.05418405]), array([0.02418405]), array([-0.00581595])]]


In [181]:
vols = [0.15,0.16,0.17,0.18,0.19,0.20]
#vols = [0.15,0.13,0.1]
print(Trees_obj.BDT(vols))

[[0.050000000000000044], [0.0690472267897135, 0.051151443693362685], [0.0837001761653265, 0.05956076855574133, 0.04238324593180678], [0.11020840540185345, 0.07539734136769051, 0.05158190125868974, 0.035288943737230856], [0.14007713406547567, 0.09167574218858708, 0.0599986697464829, 0.039267098202949435, 0.025698986457453252], [0.19436303013938055, 0.12168861787732951, 0.07618794433424712, 0.0477004584580768, 0.029864747723452205, 0.0186979996716242]]


In [162]:
0.9323249956740813/0.9372013495066713

0.9947968984091232

In [118]:
np.sqrt(6)*0.16

0.39191835884530846